Jan 13 ,Wen, 2021 

In [1]:
Jan 13 ,Wen, 2021 from pyspark.sql import  SQLContext 
from pyspark import SparkContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

C:\Users\master4\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
sc = SparkContext('local')
sqlctx = SQLContext(sc)

# 로지스틱 회귀
- **분류**
- 이산형 변수여야 함

# 타이타닉 데이터
- 생존 여부를 예측
- feature
    - Age Family_Size Sex_index Pclass

In [3]:
titanic_df = sqlctx.read.csv('../data/data1/titanic1.csv', header=True, inferSchema=True)
titanic_df.show(5)

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|          0.0|


In [9]:
f = VectorAssembler( inputCols=['Pclass', 'Age' ,'Sex_index', 'Family_Size'] ,outputCol='features' ) 
v_df = f.transform( titanic_df )
v_df = v_df.select('Survived','features')
v_df.show()

+--------+------------------+
|Survived|          features|
+--------+------------------+
|       0|[3.0,22.0,0.0,1.0]|
|       1|[1.0,38.0,1.0,1.0]|
|       1|[3.0,26.0,1.0,0.0]|
|       1|[1.0,35.0,1.0,1.0]|
|       0|[3.0,35.0,0.0,0.0]|
|       0|[3.0,33.0,0.0,0.0]|
|       0|[1.0,54.0,0.0,0.0]|
|       0| [3.0,2.0,0.0,4.0]|
|       1|[3.0,27.0,1.0,2.0]|
|       1|[2.0,14.0,1.0,1.0]|
|       1| [3.0,4.0,1.0,2.0]|
|       1|[1.0,58.0,1.0,0.0]|
|       0|[3.0,20.0,0.0,0.0]|
|       0|[3.0,39.0,0.0,6.0]|
|       0|[3.0,14.0,1.0,0.0]|
|       1|[2.0,55.0,1.0,0.0]|
|       0| [3.0,2.0,0.0,5.0]|
|       1|[2.0,33.0,0.0,0.0]|
|       0|[3.0,31.0,1.0,1.0]|
|       1|[3.0,36.0,1.0,0.0]|
+--------+------------------+
only showing top 20 rows



In [24]:
train_df, test_df = v_df.randomSplit( [0.8,0.2] ) # 7:3
train_df.count() , test_df.count()

(710, 181)

In [26]:
lr = LogisticRegression( featuresCol= 'features',
                        labelCol = 'Survived',
                        maxIter = 100                       
                           )
lr_model = lr.fit( train_df ) # 학습 시키기

In [32]:
lr_model.coefficients

DenseVector([-1.1188, -0.0402, 2.8246, -0.3167])

In [33]:
lr_model.intercept

2.4740304307836007

In [46]:
lr_model

LogisticRegressionModel: uid = LogisticRegression_b48c740c2da6, numClasses = 2, numFeatures = 4

# probability

In [45]:
pd = lr_predict.select('probability').toPandas()
pd # 0으로 분류될 확률, 1으로 분류될 확률

,probability
0,"[0.3842687030258891, 0.6157312969741109]"
1,"[0.5897069548279652, 0.4102930451720349]"
2,"[0.5315626853501401, 0.4684373146498599]"
3,"[0.4926076721746986, 0.5073923278253014]"
4,"[0.4926076721746986, 0.5073923278253014]"
...,...
176,"[0.8772808549005682, 0.1227191450994319]"
177,"[0.8973196281705009, 0.10268037182949896]"
178,"[0.378463461727147, 0.6215365382728529]"
179,"[0.4552711451928462, 0.5447288548071537]"


# prediction

In [34]:
lr_predict = lr_model.transform( test_df ) # transformation : 연산함수
lr_predict.show()
# 0.5보다 크면 0으로 예측하고, 0.5보다 작으면 1으로 예측

+--------+------------------+--------------------+--------------------+----------+
|Survived|          features|       rawPrediction|         probability|prediction|
+--------+------------------+--------------------+--------------------+----------+
|       0|[1.0,22.0,0.0,0.0]|[-0.4714686068151...|[0.38426870302588...|       1.0|
|       0|[1.0,27.0,0.0,2.0]|[0.36275407806547...|[0.58970695482796...|       0.0|
|       0|[1.0,29.0,0.0,1.0]|[0.12641883865996...|[0.53156268535014...|       0.0|
|       0|[1.0,33.0,0.0,0.0]|[-0.0295714660633...|[0.49260767217469...|       1.0|
|       0|[1.0,33.0,0.0,0.0]|[-0.0295714660633...|[0.49260767217469...|       1.0|
|       0|[1.0,40.0,0.0,0.0]|[0.25163580532410...|[0.56257908729249...|       0.0|
|       0|[1.0,49.0,0.0,2.0]|[1.24654835956905...|[0.77670179385308...|       0.0|
|       0|[1.0,50.0,0.0,1.0]|[0.97004065282247...|[0.72512760068550...|       0.0|
|       0|[1.0,50.0,1.0,0.0]|[-2.1711957080292...|[0.10236710987306...|       1.0|
|   

In [37]:
lr_predict.where( lr_predict['Survived'] == lr_predict['prediction']).count()

149

In [47]:
lr_predict.count()

181

In [42]:
print( '정확도', lr_predict.where( lr_predict['Survived'] == lr_predict['prediction']).count()/lr_predict.count())

정확도 0.8232044198895028
